# OddsJam EV Filter Analysis — Preprocessing + Initial EDA

## Purpose
This notebook ingests raw OddsJam Bet Tracker exports, performs cleaning/standardization, and produces initial exploratory analysis to validate data quality and guide downstream notebooks.

## Outputs
- Cleaned, analysis-ready bet-level dataset written to `data/processed/`
- Filter-level summary tables (profit, ROI, bet counts)
- Diagnostic plots (expected vs actual ROI, cumulative profit, bet timing patterns)

## Notes / Assumptions
- Timestamps are normalized for plotting (tz-naive for Matplotlib)
- Profit is assumed to be in `bet_profit` and stake in `stake`
- Saved filter names are standardized into `saved_filter_names`


In [2]:
import platform
import sys

sys.executable, platform.python_version()

('/Users/jniederer/projects/betting/oddsjam-ev-filters/.venv/bin/python',
 '3.11.7')

In [ ]:
# --- Standard library ---
import datetime as dt
from pathlib import Path

import matplotlib.pyplot as plt

# --- Third-party ---
import numpy as np
import pandas as pd

from oddsjam_ev.io.load import load_bet_tracker_csv
from oddsjam_ev.metrics.odds import (
    american_to_multiplier,
    american_to_prob,
    compute_ev,
    compute_ev_roi,
)
from oddsjam_ev.plotting import (
    AvgBetsByTimeBucketPlotConfig,
    # bet activity
    BetCountOverTimePlotConfig,
    # EV bucket chart
    CombinedEVPlotConfig,
    CumProfitPlotConfig,
    # profit by group / sport
    ProfitByGroupPlotConfig,
    combined_ev_summary_plot,
    plot_avg_bets_by_time_bucket,
    plot_bet_count_over_time,
    plot_cum_profit_by_group,
    plot_cum_profit_top_filters,
    plot_filter_roi_scatter,
    plot_profit_by_group_for_filters,
)

: 

: 

In [ ]:
AS_OF_DATE = "02-06-2026"
CURRENT_DT = dt.datetime.today().strftime("%m-%d-%Y")
print(f"AS_OF_DATE: {AS_OF_DATE}, CURRENT_DT: {CURRENT_DT}")

: 

: 

In [ ]:
%%time

DATA_DIR = Path("..") / "data" / "raw"
CSV_PATH = DATA_DIR / f"oddsjam-bet-tracker-{AS_OF_DATE}.csv"

df_oddsjam_bet_tracker = load_bet_tracker_csv(CSV_PATH)
df_oddsjam_bet_tracker.head()

: 

: 

In [ ]:
df_oddsjam_bet_tracker.info()

: 

: 

In [ ]:
pd.set_option("display.max_columns", None)  # show all columns
pd.set_option("display.max_colwidth", None)  # don’t truncate column values
pd.set_option("display.width", None)  # let Pandas use full available width

: 

: 

In [ ]:
CUTOFF_DATE = pd.Timestamp("2025-03-22", tz="America/New_York")

: 

: 

In [ ]:
# created_at is already datetime64[ns] (naive). Localize it to ET.
df_oddsjam_bet_tracker["created_at_est"] = df_oddsjam_bet_tracker["created_at"].dt.tz_localize(
    "America/New_York", nonexistent="shift_forward", ambiguous="infer"
)

df_filtered = df_oddsjam_bet_tracker[df_oddsjam_bet_tracker["created_at_est"] >= CUTOFF_DATE]
df_filtered = df_filtered.copy()  # avoid SettingWithCopyWarning for later edits

print(
    "Parsed OK:",
    df_oddsjam_bet_tracker["created_at_est"].notna().sum(),
    "of",
    len(df_oddsjam_bet_tracker),
)
print(
    "Min/Max:",
    df_oddsjam_bet_tracker["created_at_est"].min(),
    "→",
    df_oddsjam_bet_tracker["created_at_est"].max(),
)

df_filtered.head()

: 

: 

In [ ]:
# If df_filtered came from filtering, make it a real copy once:
df_filtered = df_filtered.copy()

stake_num = pd.to_numeric(df_filtered["stake"], errors="coerce")
mask = df_filtered["saved_filter_names"].isna() & stake_num.le(20)

df_filtered.loc[mask, "saved_filter_names"] = "TEST - Fantasy Optimizer"
print("Rows updated:", int(mask.sum()))

: 

: 

In [ ]:
# Ensure 'No Filter' label is present for null filters
df_filtered["saved_filter_names"] = df_filtered["saved_filter_names"].fillna("No Filter")

no_filter_df = df_filtered.loc[df_filtered["saved_filter_names"].eq("No Filter")].copy()
print("No Filter rows:", len(no_filter_df))

stake_num = pd.to_numeric(no_filter_df["stake"], errors="coerce")
profit_num = pd.to_numeric(no_filter_df["bet_profit"], errors="coerce")

no_filter_summary = {
    "min_date": no_filter_df["created_at_est"].min(),
    "max_date": no_filter_df["created_at_est"].max(),
    "total_stake": float(stake_num.sum(skipna=True)),
    "total_profit": float(profit_num.sum(skipna=True)),
}

for k, v in no_filter_summary.items():
    print(f"{k}: {v}")

: 

: 

In [ ]:
# Ensure labels are present
df_filtered["saved_filter_names"] = df_filtered["saved_filter_names"].fillna("No Filter")

# Keep only filters with >= 100 bets
counts = df_filtered["saved_filter_names"].value_counts(dropna=False)
keep = counts[counts >= 100].index
df_filtered = df_filtered[df_filtered["saved_filter_names"].isin(keep)].copy()

print("Remaining filters:", df_filtered["saved_filter_names"].nunique())
df_filtered["saved_filter_names"].value_counts()

: 

: 

In [ ]:
df_filtered["sportsbook"].value_counts()

: 

: 

In [ ]:
# Probabilities from placed odds vs CLV odds
df_filtered["prob_odds"] = df_filtered["odds"].apply(american_to_prob)
df_filtered["prob_clv"] = df_filtered["clv"].apply(american_to_prob)

# Profit multiplier
df_filtered["odds_multiplier"] = df_filtered["odds"].apply(american_to_multiplier)

# Expected value
df_filtered["ev"] = compute_ev(
    stake=df_filtered["stake"],
    prob=df_filtered["prob_clv"],
    odds_multiplier=df_filtered["odds_multiplier"],
)

df_filtered["ev_roi"] = compute_ev_roi(
    ev=df_filtered["ev"],
    stake=df_filtered["stake"],
)
df_filtered = df_filtered.copy()

# Sanity check
df_filtered[["odds", "clv", "prob_odds", "prob_clv", "ev", "ev_roi"]].head()

: 

: 

In [ ]:
# --- Summarize status behavior ---
status_summary = (
    df_filtered.assign(profit_known=df_filtered["bet_profit"].notna())
    .groupby("status")
    .agg(
        n_bets=("stake", "count"),
        profit_known_rate=("profit_known", "mean"),
        avg_profit=("bet_profit", "mean"),
        avg_stake=("stake", "mean"),
    )
    .sort_values("n_bets", ascending=False)
)

display(status_summary)

# --- Explicitly exclude non-settled statuses ---
NON_SETTLED_STATUSES = {
    "pending",
    "open",
    "unsettled",
}

# --- Infer settled statuses ---
settled_statuses = [
    s
    for s in status_summary.index
    if (status_summary.loc[s, "profit_known_rate"] > 0.95 and s.lower() not in NON_SETTLED_STATUSES)
]

print("\nInferred settled statuses (excluding pending):")
print(settled_statuses)

# --- Filter settled dataframe ---
df_settled = df_filtered[df_filtered["status"].isin(settled_statuses)].copy()

print("\nSettled df shape:", df_settled.shape)

: 

: 

In [ ]:
# Only keep settled bets
df_filtered = df_filtered[df_filtered["status"].isin(settled_statuses)].copy()

# Create actual ROI safely
df_filtered["actual_roi"] = np.where(
    df_filtered["stake"] > 0,
    df_filtered["bet_profit"] / df_filtered["stake"],
    np.nan,
)

# Aggregate performance by saved_filter_names
filter_summary = df_filtered.groupby("saved_filter_names", dropna=False).agg(
    total_ev_dollars=("ev", "sum"),
    total_actual_profit=("bet_profit", "sum"),
    total_stake=("stake", "sum"),
    n_bets=("stake", "count"),
    avg_ev_roi=("ev_roi", "mean"),  # unweighted mean per bet
    avg_actual_roi=("actual_roi", "mean"),  # unweighted mean per bet
)

# Weighted ROI calculations (headline)
filter_summary["total_ev_roi"] = np.where(
    filter_summary["total_stake"] > 0,
    filter_summary["total_ev_dollars"] / filter_summary["total_stake"],
    np.nan,
)
filter_summary["total_actual_roi"] = np.where(
    filter_summary["total_stake"] > 0,
    filter_summary["total_actual_profit"] / filter_summary["total_stake"],
    np.nan,
)

filter_summary_sorted = filter_summary.sort_values("total_actual_profit", ascending=False)

filter_summary_sorted.head(20).style.format(
    {
        "total_ev_dollars": "${:,.2f}",
        "total_actual_profit": "${:,.2f}",
        "total_stake": "${:,.2f}",
        "avg_ev_roi": "{:.2%}",
        "avg_actual_roi": "{:.2%}",
        "total_ev_roi": "{:.2%}",
        "total_actual_roi": "{:.2%}",
        "n_bets": "{:,.0f}",
    }
)

: 

: 

In [ ]:
ax = plot_filter_roi_scatter(filter_summary_sorted)  # defaults match original plot
plt.tight_layout()
plt.show()

: 

: 

In [ ]:
cfg = CombinedEVPlotConfig(
    bin_size=0.5,
    min_x_pct=2.0,
    max_x_pct=6.5,
    y_lim_pct=20,
    min_bets_per_bucket=50,
    show_ev_dollar_labels=True,
    clv_weight="stake",  # "stake" or "equal"
    bucket_edge="left",  # "left" or "right"
)

combined_ev_summary_plot(
    df=df_filtered,
    filter_name="PROD - Exchanges: Core Exchange Conservative (Low Risk, Slow Build)",
    cfg=cfg,
)

: 

: 

In [ ]:
N = 10
cfg = CumProfitPlotConfig(resample=None, min_points_per_line=2)

plot_cum_profit_top_filters(
    df=df_filtered,
    filter_summary=filter_summary,
    top_n=10,
    sort_col="total_actual_profit",
    cfg=cfg,
)

: 

: 

In [ ]:
N = 3
top_filters = (
    filter_summary.sort_values("total_actual_profit", ascending=False).head(N).index.tolist()
)

top_filters[:3], len(top_filters)

: 

: 

In [ ]:
plot_cum_profit_by_group(
    df=df_filtered,
    group_col="sportsbook",
    filters=top_filters,
    cfg=cfg,
)

: 

: 

In [ ]:
cfg = ProfitByGroupPlotConfig(min_bets_per_group=50, top_k=15, combine_filters=False)

plot_profit_by_group_for_filters(
    df=df_filtered,
    group_col="sport",
    filters=top_filters,
    cfg=cfg,
)

: 

: 

In [ ]:
cfg = BetCountOverTimePlotConfig(resample="W")
plot_bet_count_over_time(df=df_filtered, filters=top_filters, cfg=cfg)

: 

: 

In [ ]:
cfg = AvgBetsByTimeBucketPlotConfig(bucket="dow")  # dow of week
plot_avg_bets_by_time_bucket(df=df_filtered, filters=top_filters, cfg=cfg)

: 

: 

In [ ]:
out_dir = Path("data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / f"oddsjam-bet-tracker-processed-{AS_OF_DATE}.parquet"
df_filtered.to_parquet(out_path, index=False)

print(f"Wrote: {out_path.resolve()}")

: 

: 

: 

: 